In [8]:
import json

In [9]:
with open("../data/ffhq-dataset-v2.json","r") as infile:
    load = json.load(infile)

In [10]:
load['3242'].keys()

dict_keys(['category', 'metadata', 'image', 'thumbnail', 'in_the_wild'])

In [16]:
load['3242']

{'category': 'training',
 'metadata': {'photo_url': 'https://www.flickr.com/photos/7952984@N02/11026347713/',
  'photo_title': '',
  'author': 'suran2007',
  'country': '',
  'license': 'Attribution License',
  'license_url': 'https://creativecommons.org/licenses/by/2.0/',
  'date_uploaded': '2013-11-24',
  'date_crawled': '2018-10-10'},
 'image': {'file_url': 'https://drive.google.com/uc?id=16umbkObKDcEC9rWTExiUliBKkTF45SJy',
  'file_path': 'images1024x1024/03000/03242.png',
  'file_size': 1510761,
  'file_md5': '1c02a4b4c3aeb9df63783c0840889084',
  'pixel_size': [1024, 1024],
  'pixel_md5': 'a0dd45f1bb7f513dea41d50da42a6596',
  'face_landmarks': [[376.21, 497.23],
   [361.15, 552.88],
   [354.52, 610.86],
   [367.46, 668.29],
   [389.01, 729.67],
   [408.78, 804.34],
   [421.9, 878.1],
   [440.58, 943.08],
   [498.92, 967.65],
   [589.74, 960.81],
   [690.79, 928.29],
   [792.19, 884.29],
   [885.51, 826.47],
   [949.03, 750.71],
   [965.88, 652.35],
   [960.29, 558.12],
   [951.29, 

In [24]:
data

<Response [200]>

In [23]:
import requests

data = requests.get(load['3242']['image']['file_url'])
with open("tmp.png","wb") as outfile:
    #for content in data.content:
    outfile.write(data.content)
    outfile.close()

In [21]:
with open("tmp","wb") as outfile:
    #for content in data.content:
    outfile.write(data.content)
    outfile.close()